# 1 Import Packages

In [10]:
import xarray as xr
import cftime
import importlib
import numpy as np
import dask.distributed
from distributed import LocalCluster
import multiprocessing
import logging

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

## 1.1 Load Custom packages

In [11]:
from climtools import stat
from climtools import utils
from climtools import temporal

## 1.2 Local Dask Cluster

In [12]:
n_cores = multiprocessing.cpu_count()

In [13]:
client = dask.distributed.Client()
client

/home/bjoern/anaconda3/envs/climtools_2/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40321 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:38159 Dashboard: http://127.0.0.1:40321/status,Cluster Workers: 4 Cores: 8 Memory: 16.44 GB


# 2 Load Data

In [14]:
data_1d = xr.open_dataset("../data/tas_day_MPI-ESM1-2-LR_historical_r10i1p1f1_gn_18500101-18691231.nc", use_cftime=True, chunks = {"time":100})
data_1m = xr.load_dataset("../data/tas_mon_MPI-ESM1-2-LR_historical_r10i1p1f1_gn_18500101-18691231.nc", use_cftime=True)
data_1y = xr.load_dataset("../data/tas_yea_MPI-ESM1-2-LR_historical_r10i1p1f1_gn_18500101-18691231.nc", use_cftime=True)

# 2 Generate Data

In [6]:
start = cftime.datetime(1850,1,1,0,0,0)
end = cftime.datetime(2100,1,1,0,0,0)


In [7]:
seasonal_cycle_1d = stat.gen_seasonal_cycle(start, end, frequency = "day")

In [8]:
start = cftime.datetime(1500,1,1,0,0,0)
end = cftime.datetime(3000,1,1,0,0,0)


In [9]:
test_timeseries_yearly = stat.generate_timeseries(start, end,"year")

In [44]:
def is_leap(year):
    try:
        cftime.datetime(year=year,month=2,day=29,second=0, calendar="proleptic_gregorian")
    except:
        return False
    else:
        return True
         

In [46]:
is_leap(1852)

True

In [29]:
test_timeseries_yearly.time

<xarray.DataArray 'time' (time: 1499)>
array([cftime.DatetimeGregorian(1501, 7, 1, 12, 0, 0, 0),
       cftime.DatetimeGregorian(1502, 7, 1, 12, 0, 0, 0),
       cftime.DatetimeGregorian(1503, 7, 1, 12, 0, 0, 0), ...,
       cftime.DatetimeGregorian(2997, 7, 1, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2998, 7, 1, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2999, 7, 1, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1501-07-01 12:00:00 ... 2999-07-01 12:00:00

# 3 downsample data

In [11]:
%%time
test_mono_timeseries_1M = temporal.temporal_downsampling(test_mono_timeseries_1D , target_resolution="month").compute()

INFO:Timedelta day identified
INFO:Dependent variables ['values', 'time_bnds'] Independent Variables []


CPU times: user 4.1 s, sys: 75.1 ms, total: 4.17 s
Wall time: 4.08 s


In [23]:
xr.testing.assert_equal(test_mono_timeseries_1D.time.dt.dayofyear,test_mono_timeseries_1D.time.dt.dayofyear)

In [18]:
test_mono_timeseries_1D.time.dt.dayofyear == test_mono_timeseries_1D["values"]

<xarray.DataArray (time: 91311)>
array([ True,  True,  True, ...,  True,  True,  True])
Coordinates:
  * time     (time) object 1850-01-01 12:00:00 ... 2099-12-31 12:00:00

In [12]:
test_mono_timeseries_1M

<xarray.Dataset>
Dimensions:    (bnds: 2, time: 3000)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2099-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables:
    values     (time) float64 16.0 45.5 75.0 105.5 ... 258.5 289.0 319.5 350.0
    time_bnds  (bnds, time) object 1850-01-01 00:00:00 ... 2100-01-01 00:00:00

In [12]:
temporal.temporal_downsampling(data_1m, target_resolution="year")

INFO:Timedelta month identified
INFO:Dependent variables ['time_bnds', 'tas'] Independent Variables ['lon_bnds', 'lat_bnds']


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lon: 192, time: 20)
Coordinates:
  * time       (time) object 1850-07-02 12:00:00 ... 1869-07-02 12:00:00
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    tas        (time, lat, lon) float64 223.7 223.7 223.6 ... 255.0 255.0 255.0
    time_bnds  (bnds, time) object 1850-01-01 00:00:00 ... 1870-01-01 00:00:00
    lon_bnds   (lon, bnds) float64 -0.9375 0.9375 0.9375 ... 357.2 357.2 359.1
    lat_bnds   (lat, bnds) float64 -89.5 -87.65 -87.65 ... 87.65 87.65 89.5

# Test loaded Data

In [20]:
data_1y_cal = temporal.temporal_downsampling(data_1d,target_resolution="year").compute()

INFO:Timedelta day identified
INFO:Dependent variables ['time_bnds', 'tas'] Independent Variables ['lat_bnds', 'lon_bnds']


In [23]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [34]:
lat = data_1y.lat
lon = data_1y.lon

In [39]:
fig, ax = plt.subplots(1,1, figsize=(20,20), subplot_kw= {"projection":ccrs.PlateCarree()})

ax.pcolormesh(lon, lat, data_1y.isel(time=0)["tas"])

In [24]:
data_1y_cal

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 96, lon: 192, time: 20)
Coordinates:
  * time       (time) object 1850-07-02 12:00:00 ... 1869-07-02 12:00:00
  * lat        (lat) float64 -88.57 -86.72 -84.86 -83.0 ... 84.86 86.72 88.57
  * lon        (lon) float64 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    tas        (time, lat, lon) float32 223.67534 223.651 ... 255.02603
    time_bnds  (bnds, time) object 1850-01-01 00:00:00 ... 1870-01-01 00:00:00
    lat_bnds   (lat, bnds) float64 -89.5 -87.65 -87.65 ... 87.65 87.65 89.5
    lon_bnds   (lon, bnds) float64 -0.9375 0.9375 0.9375 ... 357.2 357.2 359.1